In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Первый взгляд на данные

In [ ]:
df_train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train.info()
print('-' * 100)
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

In [ ]:
# пропуски есть
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
# дубликатов нет
df_train.duplicated().sum()

In [ ]:
df_test.duplicated().sum()

# 2. EDA и первичная обработка данных

In [ ]:
# сразу удалим признак Id (он совпадает с индексом)
df_train = df_train.drop(columns='Id')
df_test = df_test.drop(columns='Id')

In [ ]:
# посмотрим на распределение целевой переменной
# распределение похоже на нормальное
sns.histplot(df_train['SalePrice'], kde=True);

In [ ]:
# оценим корреляцию между числовыми признаками
df_train.corr(numeric_only=True)[(df_train.corr(numeric_only=True).abs() > 0.6)]
# некоторые признаки сильно коррелируют между собой по понятным причинам

In [ ]:
df_train.corr(numeric_only=True)['SalePrice'][df_train.corr(numeric_only=True)['SalePrice'].abs() > 0.6]
# целевая переменная имеет сущ. корреляцию со следующими признаками

**2.1 Категориальные признаки**

In [ ]:
# выберем категориальные признаки - 43 всего
df_train_cat = df_train.select_dtypes(include='object')
df_train_cat.info()

In [ ]:
# добавим целевую переменную для построения графиков
df_train_cat_target = df_train_cat.join(df_train['SalePrice'])

def cat_features_vis(X):
    fig, ax = plt.subplots(9, 5, figsize=(20, 25))
    ax = ax.flatten()
    for i, col in enumerate(X.drop(columns='SalePrice').columns):
        g = sns.barplot(x=X[col], y=X['SalePrice'], ax=ax[i], estimator='mean');
        g.set_xticklabels(g.get_xticklabels(), rotation=90) # поворот меток оси для массива ax
        ax[i].set_title(col)

    plt.tight_layout()


cat_features_vis(df_train_cat_target)

In [ ]:
# найдем процент отсутствующих данных для каждого категориального признака
srs = df_train_cat.isnull().sum() / len(df_train_cat)
srs

In [ ]:
# получим список признаков с пропусками
srs[srs > 0].index.tolist()

В документации к датасету указано, что для признаков ниже (у которых есть пропуски) значение NA (либо None) означает отсутствие данной характеристики дома:

Alley, MasVnrType, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, FireplaceQu, GarageType, GarageFinish, GarageQual, GarageCond, PoolQC, Fence, MiscFeature.

Т.е. у всех этих признаков NA можно заменить на 0 или другое схожее по смыслу значение.

Единственным признаком, где не указано про NA, остался Electrical - заполним пропуски там модой (пропусков там почти нет)

In [ ]:
df_train_cat['Electrical'] = df_train_cat['Electrical'].fillna(df_train_cat['Electrical'].mode()[0])
df_train_cat = df_train_cat.fillna('0') # заполняем значением '0', чтобы оценить, как изменятся графики
df_train_cat.head()

In [ ]:
# все пропуски заполнены
df_train_cat.isnull().sum() / len(df_train_cat)

In [ ]:
# посмотрим теперь на графики
df_train_cat_target = df_train_cat.join(df_train['SalePrice'])
cat_features_vis(df_train_cat_target)

Попробуем взять большое количество признаков, при этом следует избегать возможной мультиколлинеарности (т.е. вряд ли стоит использовать одновременно ExterQual и ExterCond)

In [ ]:
# на основании графиков выберем признаки, которые существенно влияют на целевую переменную
cat_features = ['MSZoning','Street','Alley','Utilities','Neighborhood','Condition1','Condition2','HouseStyle',
'Exterior1st','Exterior2nd','MasVnrType','ExterQual','Foundation','BsmtQual','BsmtCond',
'Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','FireplaceQu','GarageType',
'GarageFinish','PoolQC','MiscFeature','SaleType','SaleCondition']

**2.2 Количественные признаки**

In [ ]:
# выберем количественные признаки - 37 всего (с учетом целевой переменной, Id ранее удалили)
df_train_num = df_train.select_dtypes(include='number')
df_train_num.info()

In [ ]:
def num_features_vis(X):
    fig, ax = plt.subplots(8, 5, figsize=(20, 25))
    ax = ax.flatten()
    for i, col in enumerate(X.drop(columns='SalePrice').columns):
        sns.scatterplot(x=X[col], y=X['SalePrice'], ax=ax[i]);
        ax[i].set_title(col)

    plt.tight_layout()


num_features_vis(df_train_num)

In [ ]:
# еще раз посмотрим на коэф. корреляции
df_train_num.corr()['SalePrice'][df_train_num.corr()['SalePrice'].abs() > 0.5]

In [ ]:
# на графиках scatterplot видны дома, которые являются выбросами
# попробуем убрать их и посмотреть, как изменится коэф. корреляции
outliers = df_train_num[df_train_num['LotFrontage'] > 300].index
df_train.loc[outliers]

In [ ]:
df_train_num[['LotFrontage', 'SalePrice']].corr()

In [ ]:
df_train_num.drop(index=outliers)[['LotFrontage', 'SalePrice']].corr()
# коэф. корреляции вырос, но не существенно

In [ ]:
# выберем признаки, которые имеют коэф. корреляции по модулю не менее 0.5 с SalePrice (в том числе и сама SalePrice)
num_featues = df_train_num.corr()['SalePrice'][df_train_num.corr()['SalePrice'].abs() > 0.5].index.tolist()
num_featues

Попробуем взять большое количество признаков, при этом следует избегать возможной мультиколлинеарности

In [ ]:
# добавим некоторые признаки (на основе scatterplot), которые могут быть полезны
num_featues += ['MSSubClass', 'LotFrontage', 'LotArea', '2ndFlrSF']
num_featues

In [ ]:
# очевидно, что GarageCars и GarageArea сильно коррелируют между собой
# поэтому удалим один из них
num_featues.remove('GarageCars')

In [ ]:
df_train[num_featues].corr()

In [ ]:
# также удалим TotalBsmtSF (коррелирует с 1stFlrSF)
# и TotRmsAbvGrd (коррелирует с GrLivArea)
# целевую переменную отсюда уберем
num_featues.remove('TotalBsmtSF')
num_featues.remove('TotRmsAbvGrd')
num_featues.remove('SalePrice')
num_featues

In [ ]:
# признак MSSubClass является количественным дискретным - по сути, 
# категориальным номинальным - поэтому сразу добавим его в cat_features
sns.barplot(x=df_train_num['MSSubClass'], y=df_train_num['SalePrice']);

In [ ]:
num_featues.remove('MSSubClass')
cat_features.append('MSSubClass')

**Выводы**

Имеется большое количество признаков, многие из которых важны. Единственно правильного варианта выделения признаков здесь нет, вариантов много. Попробуем взять большое число признаков

# 3. Обработка данных

Будем работать отдельно с категориальными и количественными признаками, как и ранее

In [ ]:
# разделим данные на признаки и таргет
X_train_cat = df_train[cat_features]
X_train_num = df_train[num_featues]
y_train = df_train['SalePrice']

X_test_cat = df_test[cat_features]
X_test_num = df_test[num_featues]

**3.1 Обработка пропусков**

**Категориальные признаки**

In [ ]:
X_train_cat.isnull().sum()

In [ ]:
X_test_cat.isnull().sum()

Как указывалось в разделе EDA, для большинства признаков пропуск означает отсутствие данной характеристики дома. Поэтому для них всех (кроме Electrical, Utilities, KitchenQual, Functional и SaleType), заполним пропуски 0, для остальных - заполним модой (там почти нет пропусков).

Примечание - используем 0 как строку, а не число, т.к. в дальнейшем при кодировании данные должны быть единообразны

In [ ]:
X_train_cat.loc[:, 'Electrical'] = X_train_cat['Electrical'].fillna(X_train_cat['Electrical'].mode()[0])
X_train_cat = X_train_cat.fillna('0')

X_test_cat.loc[:, 'Utilities'] = X_test_cat['Utilities'].fillna(X_test_cat['Utilities'].mode()[0])
X_test_cat.loc[:, 'KitchenQual'] = X_test_cat['KitchenQual'].fillna(X_test_cat['KitchenQual'].mode()[0])
X_test_cat.loc[:, 'Functional'] = X_test_cat['Functional'].fillna(X_test_cat['Functional'].mode()[0])
X_test_cat.loc[:, 'SaleType'] = X_test_cat['SaleType'].fillna(X_test_cat['SaleType'].mode()[0])
X_test_cat = X_test_cat.fillna('0')

In [ ]:
# пропусков не осталось
X_train_cat.isnull().sum().sum(), X_test_cat.isnull().sum().sum()

**Количественные признаки**

In [ ]:
X_train_num.isnull().sum()

In [ ]:
X_test_num.isnull().sum()

Можно предположить, что пропуск в данном случае также означает отсутствие данной характеристики дома (Lot Frontage - длина улицы, примыкающей к дому; GarageArea - площадь гаража). Поэтому заполним все нулями

In [ ]:
X_train_num = X_train_num.fillna(0)
X_test_num = X_test_num.fillna(0)

In [ ]:
# пропусков не осталось
X_train_num.isnull().sum().sum(), X_test_num.isnull().sum().sum()

**3.2 Работа с выбросами**

In [ ]:
# оценим процент выбросов для каждого признака через IQR
def outlier_count(X):
    Q25, Q75 = X.quantile(0.25, axis=0), X.quantile(0.75, axis=0)
    IQR = Q75 - Q25
    lower_bound, upper_bound = Q25 - 1.5 * IQR, Q75 + 1.5 * IQR

    # доля выбросов для каждого признака в процентах
    quantity = ((X >= upper_bound) | (X <= lower_bound)).sum(axis=0) / X.shape[0] * 100
    return quantity.sort_values(ascending=False)


outlier_count(X_train_num)
# выбросов не так много

In [ ]:
# посмотрим на выбросы
def outliers_vis(X):
    fig, ax = plt.subplots(2, 5, figsize=(18, 6))
    ax = ax.flatten()
    for i, col in enumerate(X.columns):
        sns.boxplot(x=X[col], ax=ax[i]);
        ax[i].set_title(col)

    plt.tight_layout()


outliers_vis(X_train_num)

In [ ]:
# выберем признаки, для которых есть смысл удалять выбросы
# это признаки, связанные с площадью или размерами (см. boxplot и scatterplot выше)
features_with_outliers = ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea', 'GarageArea']

def outlier_remove(X):
    Q25, Q75 = X.quantile(0.25, axis=0), X.quantile(0.75, axis=0)
    IQR = Q75 - Q25
    lower_bound, upper_bound = Q25 - 1.5 * IQR, Q75 + 1.5 * IQR
    
    # маска, соответствующая значениям без выбросов в данных столбцах
    mask = ((X >= lower_bound) & (X <= upper_bound)).all(axis=1)
    return mask

mask_for_outliers = outlier_remove(X_train_num[features_with_outliers])
# эту маску применим далее в случае удаления выбросов (в том числе и к X_train_cat)

**3.3 Кодирование категориальных переменных**

В данном случае некоторые категориальные признаки имеют ранжированность (это указано в документации). Поэтому есть смысл использовать это при кодировании 

In [ ]:
# учитываем, что ранее заполняли пропуски нулями (строковыми значениями)
X_train_cat.loc[:, 'Utilities'] = X_train_cat['Utilities'].map({
    'AllPub': 3, 'NoSewr': 2, 'NoSeWa': 1, 'ELO': 0})
X_train_cat.loc[:, 'ExterQual'] = X_train_cat['ExterQual'].map({
    'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0})
X_train_cat.loc[:, 'BsmtQual'] = X_train_cat['BsmtQual'].map({
    'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, '0': 0})
X_train_cat.loc[:, 'BsmtCond'] = X_train_cat['BsmtCond'].map({
    'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, '0': 0})
X_train_cat.loc[:, 'HeatingQC'] = X_train_cat['HeatingQC'].map({
    'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0})
X_train_cat.loc[:, 'CentralAir'] = X_train_cat['CentralAir'].map({
    'Y': 1, 'N': 0})
X_train_cat.loc[:, 'KitchenQual'] = X_train_cat['KitchenQual'].map({
    'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0})
X_train_cat.loc[:, 'FireplaceQu'] = X_train_cat['FireplaceQu'].map({
    'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, '0': 0})
X_train_cat.loc[:, 'GarageFinish'] = X_train_cat['GarageFinish'].map({
    'Fin': 3, 'RFn': 2, 'Unf': 1, '0': 0})
X_train_cat.loc[:, 'PoolQC'] = X_train_cat['PoolQC'].map({
    'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, '0': 0})

In [ ]:
# то же самое для test
X_test_cat.loc[:, 'Utilities'] = X_test_cat['Utilities'].map({
    'AllPub': 3, 'NoSewr': 2, 'NoSeWa': 1, 'ELO': 0})
X_test_cat.loc[:, 'ExterQual'] = X_test_cat['ExterQual'].map({
    'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0})
X_test_cat.loc[:, 'BsmtQual'] = X_test_cat['BsmtQual'].map({
    'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, '0': 0})
X_test_cat.loc[:, 'BsmtCond'] = X_test_cat['BsmtCond'].map({
    'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, '0': 0})
X_test_cat.loc[:, 'HeatingQC'] = X_test_cat['HeatingQC'].map({
    'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0})
X_test_cat.loc[:, 'CentralAir'] = X_test_cat['CentralAir'].map({
    'Y': 1, 'N': 0})
X_test_cat.loc[:, 'KitchenQual'] = X_test_cat['KitchenQual'].map({
    'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0})
X_test_cat.loc[:, 'FireplaceQu'] = X_test_cat['FireplaceQu'].map({
    'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, '0': 0})
X_test_cat.loc[:, 'GarageFinish'] = X_test_cat['GarageFinish'].map({
    'Fin': 3, 'RFn': 2, 'Unf': 1, '0': 0})
X_test_cat.loc[:, 'PoolQC'] = X_test_cat['PoolQC'].map({
    'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, '0': 0})

In [ ]:
# закодированные порядковые признаки назовем ordinal_cat_features
# остальные категориальные признаки (other_cat_features) закодируем через OneHotEncoder
ordinal_cat_features = ['Utilities', 'ExterQual', 'BsmtQual', 'BsmtCond','HeatingQC',
                        'CentralAir', 'KitchenQual', 'FireplaceQu', 'GarageFinish', 'PoolQC']
other_cat_features = list(set(cat_features) - set(ordinal_cat_features))
other_cat_features

In [ ]:
from sklearn.preprocessing import OneHotEncoder

onehotencoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
encoded_X_train_cat = pd.DataFrame(onehotencoder.fit_transform(X_train_cat[other_cat_features]))
encoded_X_train_cat.columns = onehotencoder.get_feature_names_out() # добавляем названия столбцов

X_train_cat = X_train_cat.join(encoded_X_train_cat) # присоединим новые признаки к исходному датафрейму
X_train_cat = X_train_cat.drop(columns=other_cat_features) # удалим исходные признаки

X_train_cat

In [ ]:
# то же самое для test
encoded_X_test_cat = pd.DataFrame(onehotencoder.transform(X_test_cat[other_cat_features]))
encoded_X_test_cat.columns = onehotencoder.get_feature_names_out() # добавляем названия столбцов

X_test_cat = X_test_cat.join(encoded_X_test_cat) # присоединим новые признаки к исходному датафрейму
X_test_cat = X_test_cat.drop(columns=other_cat_features) # удалим исходные признаки

X_test_cat

In [ ]:
# признаки ordinal_cat_features имеют тип object
# преобразуем их к int и сразу добавим в X_train_num, удалив из X_train_cat
X_train_num = X_train_num.join(X_train_cat[ordinal_cat_features].astype('int64'))
X_train_cat = X_train_cat.drop(columns=ordinal_cat_features)
X_train_num

In [ ]:
# то же самое для test
X_test_num = X_test_num.join(X_test_cat[ordinal_cat_features].astype('int64'))
X_test_cat = X_test_cat.drop(columns=ordinal_cat_features)
X_test_num

**3.4 Масштабирование и преобразование числовых признаков**

In [ ]:
# преобразуем признаки YearBuilt и YearRemodAdd, предполагая, что
# данные актуальны на 2016 год (указано на странице соревнования)
# также переименуем признаки
X_train_num.loc[:, ['YearBuilt', 'YearRemodAdd']] = 2016 - X_train_num[['YearBuilt', 'YearRemodAdd']]
X_test_num.loc[:, ['YearBuilt', 'YearRemodAdd']] = 2016 - X_test_num[['YearBuilt', 'YearRemodAdd']]

X_train_num = X_train_num.rename(columns={'YearBuilt': 'YearSinceBuilt', 'YearRemodAdd': 'YearSinceRemodAdd'})
X_test_num = X_test_num.rename(columns={'YearBuilt': 'YearSinceBuilt', 'YearRemodAdd': 'YearSinceRemodAdd'})

In [ ]:
# посмотрим, насколько числовые признаки близки к нормальному распределению
from scipy.stats import kstest, shapiro, skew 

def norm_distr_check(X):
    df_norm_distr = pd.DataFrame(columns=['Критерий Колмогорова-Смирнова (p-value)', 
                                          'Критерий Шапиро-Уилка (p-value)', 
                                          'Коэффициент асимметрии']
                                            ) # пустой фрейм под результаты
    for col in X.columns:
        df_norm_distr.loc[col, 'Критерий Колмогорова-Смирнова (p-value)'] = kstest(X[col], 'norm').pvalue
        df_norm_distr.loc[col, 'Критерий Шапиро-Уилка (p-value)'] = shapiro(X[col]).pvalue
        df_norm_distr.loc[col, 'Коэффициент асимметрии'] = skew(X[col])
        
    return df_norm_distr


norm_distr_check(X_train_num)
# можно сделать вывод, что все числовые признаки имеют отличное от нормального распределение

In [ ]:
# оценим распределения числовых признаков визуально
def feature_vis(X):
    fig, ax = plt.subplots(4, 5, figsize=(15, 12))
    ax = ax.flatten()

    for i, col in enumerate(X.columns):
        sns.histplot(X[col], ax=ax[i], kde=True);
        ax[i].set_title(col)

    plt.tight_layout()


feature_vis(X_train_num)

In [ ]:
# сравним различные варианты преобразований признаков
# в данном случае в датафрейме есть нулевые значения, поэтому
# используем преобразование Йео-Джонсона
from sklearn.preprocessing import PowerTransformer
from scipy.stats import skew

def feature_transform_score(X):
    transform_scores = pd.DataFrame() # пустой фрейм под результаты
    for col in X.columns:
        # по умолчанию к преобразованным данным применяется нормализация с нулевым средним и единичной дисперсией
        yeo_johnson = PowerTransformer(method='yeo-johnson', standardize=True)
        
        transform_scores.loc['Original distribution', col] = skew(X[col])
        transform_scores.loc['Yeo-Johnson transformation', col] = skew(yeo_johnson.fit_transform(X[[col]]))
        transform_scores.loc['Log transformation', col] = skew(np.log1p(X[col]))
        transform_scores.loc['Sqrt transformation', col] = skew(np.sqrt(X[col]))

    return transform_scores


transform_scores = feature_transform_score(X_train_num)
transform_scores

**Нужно ли применять преобразования к упорядоченным признакам (Utilities и пр.)?
В данном случае применяются ко всем признакам**

In [ ]:
# теперь применим соответствующее преобразование ко всем столбцам,
# чтобы добиться минимальной асимметрии
# применяем для test те же преобразования, что и для train (по тем же столбцам)

def apply_transformations(X, scores):
    # сделаем копию датафрейма, чтобы исходный датафрейм и преобразованный были независимы 
    X_transformed = X.copy()
    
    # нужны модули значений
    scores = scores.abs().idxmin(axis=0)
    
    yeo_transform_cols = scores[scores == 'Yeo-Johnson transformation'].index.tolist()
    log_transform_cols = scores[scores == 'Log transformation'].index.tolist()
    sqrt_transform_cols = scores[scores == 'Sqrt transformation'].index.tolist()

    for col in yeo_transform_cols:
        # не применяем нормализацию, т.к. к log и sqrt она не применяется
        # (чтобы потом применить Scaler ко всем признакам сразу)
        yeo_johnson = PowerTransformer(method='yeo-johnson', standardize=False)
        X_transformed.loc[:, col] = yeo_johnson.fit_transform(X_transformed[[col]])

    for col in log_transform_cols:
        X_transformed.loc[:, col] = np.log1p(X_transformed[col])
        
    for col in sqrt_transform_cols:
        X_transformed.loc[:, col] = np.sqrt(X_transformed[col])

    return X_transformed


X_train_num_transformed = apply_transformations(X_train_num, transform_scores)
X_test_num_transformed = apply_transformations(X_test_num, transform_scores)

X_train_num_transformed.head()

In [ ]:
# оценим распределения числовых признаков теперь
feature_vis(X_train_num_transformed)

In [ ]:
# полученные датафреймы также без пропусков
X_train_num_transformed.isnull().sum().sum(), X_test_num_transformed.isnull().sum().sum()

**На текущем этапе получили датафреймы X_train_num_transformed и X_test_num_transformed с преобразованными числовыми признаками, которые можно использовать в дальнейшем вместо X_train_num и X_test_num (нормализовывать и т.д.)**

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# нормализуем числовые признаки и сразу присоединим их к закодированным категориальным
# здесь можно использовать либо X_train_num и X_test_num, либо X_train_num_transformed и X_test_num_transformed
X_train_scaled = X_train_cat.join(pd.DataFrame(scaler.fit_transform(X_train_num_transformed), columns=X_train_num_transformed.columns))
X_test_scaled = X_test_cat.join(pd.DataFrame(scaler.transform(X_test_num_transformed), columns=X_test_num_transformed.columns))

In [ ]:
X_train_scaled.head()

In [ ]:
X_test_scaled.head()

In [ ]:
# финальная проверка на корректность данных
X_train_scaled.info()
print('-' * 100)
X_test_scaled.info()

In [ ]:
X_train_scaled.isnull().sum().sum(), X_test_scaled.isnull().sum().sum()

**3.5 Конструирование признаков**

В данном случае дополнительные признаки не создаются.

Попробуем применить PCA, чтобы оценить вклад признаков

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(20)
pca.fit(X_train_scaled)

# процент дисперсии, объясняемый каждым из выбранных компонентов (собств. числа ковариационной матрицы)
pca.explained_variance_ratio_
# с учетом большого числа признаков есть смысл попробовать применить PCA

**3.6 Удаление выбросов**

Удалим выбросы, использую ранее полученную маску

In [ ]:
"""
X_train_scaled = X_train_scaled[mask_for_outliers]
y_train = y_train[mask_for_outliers]
"""

# 4. Моделирование и оценка результата

**4.1 SVM**

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.svm import SVR

scores = pd.DataFrame() # пустой фрейм под все модели

def objective(trial):
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    degree = trial.suggest_int('degree', 2, 6)
    C = trial.suggest_float('C', 1e-2, 1e5, log=True)
    svm_optuna = SVR(kernel=kernel, degree=degree, C=C)
    # минимизируем MSE (RMSE), т.к. в соревновании результат оценивается через нее
    score = cross_val_score(svm_optuna, X_train_scaled, y_train, scoring='neg_mean_squared_error', cv=5).mean()
    return score


def model_score(model):
    # здесь оценим метрики регрессии
    score = cross_validate(model, X_train_scaled, y_train, cv=5, scoring=['neg_root_mean_squared_error', 'r2'])
    return {k: v.mean() for k, v in score.items()}


study = optuna.create_study(direction='maximize', study_name='SVM')
study.optimize(objective, n_trials=100)

svm_optuna_score = study.best_value
svm_optuna_params = study.best_params

scores.loc['SVM', 'Score'] = svm_optuna_score
scores.loc['SVM', 'Model_params'] = str([f'{k}: {v}' for k, v in svm_optuna_params.items()])

In [ ]:
svm_optuna_params

In [ ]:
svm = SVR(kernel='rbf', degree=2, C=99857.05637120269)
model_score(svm)

In [ ]:
svm.fit(X_train_scaled, y_train)
y_pred_svm = svm.predict(X_test_scaled)
idx = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')['Id']
submission_svm = pd.DataFrame({'Id': idx, 'SalePrice': y_pred_svm})
submission_svm.to_csv("submission_svm.csv", index=False)
print('✅ submission_svm.csv created successfully!')

# результат на test - 0.14683 без удаления выбросов
# результат на test - 0.14928 с удалением выбросов
# результат на test - 0.42613 без удаления выбросов с преобразованными данными ???

**4.2 Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 500, step=10)
    criterion = trial.suggest_categorical('criterion', ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'])
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20, step=1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 20, step=1)
    rf_optuna = RandomForestRegressor(n_estimators=n_estimators, criterion=criterion,
                                      min_samples_split=min_samples_split, 
                                      min_samples_leaf=min_samples_leaf, random_state=42)
    score = cross_val_score(rf_optuna, X_train_scaled, y_train, scoring='neg_mean_squared_error', cv=5).mean()
    return score
    

study = optuna.create_study(direction='maximize', study_name='Random Forest')
study.optimize(objective, n_trials=100)

rf_optuna_score = study.best_value
rf_optuna_params = study.best_params

scores.loc['Random Forest', 'Score'] = rf_optuna_score
scores.loc['Random Forest', 'Model_params'] = str([f'{k}: {v}' for k, v in rf_optuna_params.items()])

In [ ]:
rf_optuna_params

In [ ]:
rf = RandomForestRegressor(n_estimators=270, criterion='friedman_mse',
                           min_samples_split=3, min_samples_leaf=2,
                           random_state=42)
model_score(rf)

In [ ]:
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)
idx = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')['Id']
submission_rf = pd.DataFrame({'Id': idx, 'SalePrice': y_pred_rf})
submission_rf.to_csv("submission_rf.csv", index=False)
print('✅ submission_rf.csv created successfully!')

# результат на test - 0.15207 без удаления выбросов
# результат на test - 0.15479 с удалением выбросов
# результат на test - 0.30563 без удаления выбросов с преобразованными данными ???

**4.3 AdaBoost**

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 500, step=10)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1, log=True)
    ada_boost_optuna = AdaBoostRegressor(n_estimators=n_estimators, learning_rate=learning_rate, random_state=42)
    score = cross_val_score(ada_boost_optuna, X_train_scaled, y_train, scoring='neg_mean_squared_error', cv=5).mean()
    return score


study = optuna.create_study(direction='maximize', study_name='AdaBoost')
study.optimize(objective, n_trials=100)

adaboost_optuna_score = study.best_value
adaboost_optuna_params = study.best_params

scores.loc['AdaBoost', 'Score'] = adaboost_optuna_score
scores.loc['AdaBoost', 'Model_params'] = str([f'{k}: {v}' for k, v in adaboost_optuna_params.items()])

In [ ]:
adaboost_optuna_params

In [ ]:
adaboost = AdaBoostRegressor(n_estimators=460, learning_rate=0.09597846917600465, random_state=42)
model_score(adaboost)

In [ ]:
adaboost.fit(X_train_scaled, y_train)
y_pred_adaboost = adaboost.predict(X_test_scaled)
idx = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')['Id']
submission_adaboost = pd.DataFrame({'Id': idx, 'SalePrice': y_pred_adaboost})
submission_adaboost.to_csv("submission_adaboost.csv", index=False)
print('✅ submission_adaboost.csv created successfully!')

# результат на test - 0.21537 без удаления выбросов
# результат на test - 0.19279 с удалением выбросов
# результат на test - 0.38725 без удаления выбросов с преобразованными данными ???

**4.4 GradienBoosting**

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

def objective(trial):
    loss = trial.suggest_categorical('loss', ['squared_error', 'absolute_error', 'huber', 'quantile'])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1, log=True)
    n_estimators = trial.suggest_int('n_estimators', 100, 500, step=10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20, step=1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 20, step=1)
    max_depth = trial.suggest_int('max_depth', 2, 12, step=1)
    grad_boost_optuna = GradientBoostingRegressor(loss=loss, learning_rate=learning_rate, 
                                                  n_estimators=n_estimators, min_samples_split=min_samples_split,
                                                  min_samples_leaf=min_samples_leaf, max_depth=max_depth, 
                                                  random_state=42)
    score = cross_val_score(grad_boost_optuna, X_train_scaled, y_train, scoring='neg_mean_squared_error', cv=5).mean()
    return score


study = optuna.create_study(direction='maximize', study_name='Gradient Boosting')
study.optimize(objective, n_trials=100)

grad_boost_optuna_score = study.best_value
grad_boost_optuna_params = study.best_params

scores.loc['GradientBoosting', 'Score'] = grad_boost_optuna_score
scores.loc['GradientBoosting', 'Model_params'] = str([f'{k}: {v}' for k, v in grad_boost_optuna_params.items()])

In [ ]:
grad_boost_optuna_params

In [ ]:
grad_boost = GradientBoostingRegressor(loss='squared_error', learning_rate=0.03609481196163702,
                                       n_estimators=430, min_samples_split=3,  min_samples_leaf=3,
                                       max_depth=6, random_state=42)
model_score(grad_boost)

In [ ]:
grad_boost.fit(X_train_scaled, y_train)
y_pred_grad_boost = grad_boost.predict(X_test_scaled)
idx = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')['Id']
submission_grad_boost = pd.DataFrame({'Id': idx, 'SalePrice': y_pred_grad_boost})
submission_grad_boost.to_csv("submission_grad_boost.csv", index=False)
print('✅ submission_grad_boost.csv created successfully!')

# результат на test - 0.14770 без удаления выбросов
# результат на test - 0.14266 с удалением выбросов
# результат на test - 0.33854 без удаления выбросов с преобразованными данными ???

**4.5 CatBoost**

In [ ]:
from catboost import CatBoostRegressor

# параметры CatBoostRegressor:
# iterations - количество деревьев
# learning_rate - обычно устанавливается в пределах 0.01–0.1 
# depth - глубина деревьев

def objective(trial):
    iterations = trial.suggest_int('iterations', 100, 500, step=10)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1, log=True)
    depth = trial.suggest_int('depth', 1, 12, step=1)
    cat_boost_optuna = CatBoostRegressor(iterations=iterations, learning_rate=learning_rate,
                                         depth=depth, verbose=False, random_state=42)
    score = cross_val_score(cat_boost_optuna, X_train_scaled, y_train, scoring='neg_mean_squared_error', cv=5).mean()
    return score


study = optuna.create_study(direction='maximize', study_name='CatBoost')
study.optimize(objective, n_trials=100)

cat_boost_optuna_score = study.best_value
cat_boost_optuna_params = study.best_params

scores.loc['CatBoost', 'Score'] = cat_boost_optuna_score
scores.loc['CatBoost', 'Model_params'] = str([f'{k}: {v}' for k, v in cat_boost_optuna_params.items()])

In [ ]:
cat_boost_optuna_params

In [ ]:
cat_boost = CatBoostRegressor(iterations=400, learning_rate=0.097206691361421,
                              depth=5, verbose=False, random_state=42)
model_score(cat_boost)

In [ ]:
cat_boost.fit(X_train_scaled, y_train)
y_pred_cat_boost = cat_boost.predict(X_test_scaled)
idx = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')['Id']
submission_cat_boost = pd.DataFrame({'Id': idx, 'SalePrice': y_pred_cat_boost})
submission_cat_boost.to_csv("submission_cat_boost.csv", index=False)
print('✅ submission_cat_boost.csv created successfully!')

# результат на test - 0.13752 без удаления выбросов
# результат на test - 0.14100 с удалением выбросов
# результат на test - 0.37617 без удаления выбросов с преобразованными данными ???

In [ ]:
scores